In [1]:
import pandas as pd

In [2]:
#Loading data
df = pd.read_csv("D:\\Programming\\Python\\recommendation system\\archive\\spotify_millsongdata.csv")

KeyboardInterrupt: 

In [ ]:
df.shape

(57650, 4)

In [ ]:
#Kiểm tra các ô null
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [ ]:
df = df.drop('link', axis = 1)

In [ ]:
df.head(5)

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [ ]:
df['text'][0]

"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [ ]:
df.sample(5000)

,artist,song,text
26094,Bon Jovi,Blaze Of Glory,I wake up in the morning \r\nAnd I raise my w...
49139,Procol Harum,About To Die,"About to die, the crowds applaud you \r\nAbou..."
4834,Elton John,And The House Fell Down,The sun is up and the shades are all pulled do...
4257,Dolly Parton,A Christmas To Remember,You've made this a Christmas to remember \r\n...
50463,Reba Mcentire,I'll Give You Something To Miss,Well there's a half moon starlit sky. \r\nAll...
...,...,...,...
736,Andrea Bocelli,Sogno,SOGNO \r\n \r\nVa ti aspettero' \r\nII fior...
10718,Kirk Franklin,Hello Fear,Hello fear \r\nBefore you sit down there's so...
17395,Regine Velasquez,Langit,Tayo ay iisa \r\nkahit pa magkawalay \r\nmal...
16456,Procol Harum,So Far Behind,"Your convent-cloistered, cluttered mind \r\n[..."


In [ ]:
df = df.sample(5000)

In [ ]:
df.head(5)

,artist,song,text
16650,Quicksilver Messenger Service,What About Me,You poisoned my sweet water. \r\nYou cut down...
21483,Who,Mary-Anne With The Shaky Hands,I danced with Linda \r\nI danced with Jean \...
20186,Unseen,Scream Out,1-2-3... go! \r\n \r\nWell all my life it's ...
40423,Kinks,Good Day,The sky is blue \r\nBut there are clouds in m...
54520,Uriah Heep,Dream On,"(being, in love) \r\nBeen travelling this roa..."


Cleaning

In [ ]:
df['text']=df['text'].str.lower().replace('r^\w\s',' ').replace(r'\n',' ', regex=True)

In [ ]:
df.head(5)

,artist,song,text
16650,Quicksilver Messenger Service,What About Me,you poisoned my sweet water. \r you cut down ...
21483,Who,Mary-Anne With The Shaky Hands,i danced with linda \r i danced with jean \r...
20186,Unseen,Scream Out,1-2-3... go! \r \r well all my life it's be...
40423,Kinks,Good Day,the sky is blue \r but there are clouds in my...
54520,Uriah Heep,Dream On,"(being, in love) \r been travelling this road..."


Tokenization và Vectorization

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer


In [ ]:
stemmer = PorterStemmer()

In [ ]:
def token(txt):
    token = nltk.word_tokenize(txt) # tách chuỗi văn bản thành các từ  riêng lẻ
    a = [stemmer.stem(w) for w in token] # duyệt qua từng từ trong token và chuyển về dạng gốc
    return " ".join(a) # ghép các từ trong list a thành một chuỗi, ngăn cách bởi dấu " "

In [ ]:
token("you are beautiful")

'you are beauti'

In [ ]:
df['text'].apply(lambda x: token(x))

16650    you poison my sweet water . you cut down my gr...
21483    i danc with linda i danc with jean i danc with...
20186    1-2-3 ... go ! well all my life it 's been the...
40423    the sky is blue but there are cloud in my head...
54520    ( be , in love ) been travel thi road for so l...
                               ...                        
49963    what doe a man do what doe a man do who ca n't...
42974    burn the witch , burn the witch , do n't take ...
29903    i lie in bed besid you and watch you while you...
19695    say it say it say it tell it like it is say it...
43054    [ vers : ] i do n't know if i can open up i 'v...
Name: text, Length: 5000, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')
# sử dụng từ riêng lẻ trong văn bản để tạo vector, các option khác: char, char_wb
# stopwords: loại bỏ các từ không mang ý nghĩa như "a", "an", "the", "me", ...

In [ ]:
matrix = tfid.fit_transform(df['text'])

In [ ]:
similer = cosine_similarity(matrix)
## Kết quả là một ma trận đối xứng, giá trị ở hàng thứ i và cột thứ j thể hiện độ tương đồng cosin giữa hai vector biểu diễn văn bản i và j trong tập dữ liệu.

In [ ]:
similer[0]

array([1.        , 0.00892434, 0.02403606, ..., 0.01126776, 0.03863066,
       0.05446944])

In [ ]:
import pickle
pickle.dump(similer,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))